In [ ]:
run_tests = False

In [ ]:
import geopandas as gpd
import pandas as pd
import time

In [ ]:
# parameters specific to this notebook
ldc_fract_area_threshold = 0.001
run_intersection_tract_ldc = True # when False, loads results saved from previous run
test_urban_populations = False

In [ ]:
# input files from GIM & other GEM compilation:
prod_leak_timestamp = '2020-11-12_0933'
trans_leak_timestamp = '2020-11-30_1608'
ldc_leak_rcei_timestamp = '2020-11-30_1610'

gim_cities_file = 'GIM cities in index 2020-10-31.xlsx'
parameters_file = 'GIM parameters file.xlsx'

In [ ]:
# paths
lca_path = '/Users/masoninman/Dropbox/GEM/LCA of natural gas use/'
inputs_path = lca_path + 'US cities LCA of gas model/US gas model inputs/'
census_path = lca_path + 'US Census/'
eia_176_path = lca_path + 'EIA data for LCA of gas/EIA-176 data/'

In [ ]:
# read parameters
df = pd.read_excel(inputs_path + 'GIM parameters file.xlsx', sheet_name='main parameters')
parameters_main = df.set_index('parameter name')['parameter value']

data_year = int(parameters_main.at['data_year'])
contiguous_us_only = parameters_main.at['contiguous_us_only']
ch4_fract_in_ng_consumer_grade = parameters_main.at['ch4_fract_in_ng_consumer_grade']
ch4_kg_per_mcf = parameters_main.at['ch4_kg_per_mcf']

In [ ]:
# input files from external sources:
fips_file = 'US Census - center of population by state (2010 census) and FIPS Code.xlsx'
operations_year_file = f'EIA-176 Type of Operations and Sector Items {data_year}.xlsx'
ldc_population_served_file = 'LDC pop served and res-comm Mcf per person 2020-11-01_0946.xlsx'
state_abbrev_file = 'US states and abbreviations.xlsx'
urban_file = 'cb_2018_us_ua10_500k.shp'

In [ ]:
# standard conversions
km2_per_m2 = 1e-6
g_per_gg = 1e9
ton_per_g = 1e-6
gg_per_g = 1e-9
gg_per_kg = 1e-6

In [ ]:
# convert volume NG to mass of CH4 it contains
conversion_consumer_ng_mcf_to_ch4_gg = ch4_fract_in_ng_consumer_grade * ch4_kg_per_mcf * gg_per_kg

## Get state information (names, abbreviations, FIPS codes)

In [ ]:
def read_state_abbreviations(state_abbrev_file):
    states_df = pd.read_excel(inputs_path + state_abbrev_file)

    states_abbrev_to_full_dict = states_df.set_index('abbrev')['state'].to_dict()
    states_full_to_abbrev_dict = states_df.set_index('state')['abbrev'].to_dict()
    
    return(states_abbrev_to_full_dict, states_full_to_abbrev_dict)

In [ ]:
def read_fips_file():
    # get state codes (FIPS)
    # https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html
    fips = pd.read_excel(inputs_path + fips_file, dtype={'STATEFP': str})

    if contiguous_us_only==True:
        fips = fips.loc[~fips['STNAME'].isin(['Alaska', 'Hawaii', 'Puerto Rico'])]
    else:
        pass
    
    return fips

In [ ]:
(states_abbrev_to_full_dict, states_full_to_abbrev_dict) = read_state_abbreviations(state_abbrev_file)
fips = read_fips_file()

In [ ]:
# import state boundaries
state_bound_folder = 'cb_2018_us_state_500k'
state_bound = gpd.read_file(census_path + state_bound_folder)
state_bound = state_bound.to_crs(epsg=2163)
print(state_bound.crs)

state_bound = state_bound.rename(columns={'STUSPS': 'state abbrev'})
state_bound = state_bound.set_index('state abbrev')
state_bound = state_bound[['geometry']]

In [ ]:
def test_for_crs_mismatch(gdf_1, gdf_2):
    if gdf_1.crs['init']==gdf_2.crs['init']:
        pass
    else:
        print("ERROR!" + f" CRS mismatch: gdf_1.crs: {gdf_1.crs}; gdf_2.crs: {gdf_2.crs}")
        
    # no return

## LDCs data set from ORNL (via HIFLD)

**Notes on ORNL:**
* Data from Department of Homeland Security (DHS), [HIFLD](https://hifld-geoplatform.opendata.arcgis.com/datasets/natural-gas-local-distribution-company-service-territories)
* [Metadata page](https://www.arcgis.com/sharing/rest/content/items/a3d48c142f88433ab77aa755a56aa07a/info/metadata/metadata.xml?format=default&output=html) says: 
  * Temporal extent: Beginning date: 2014-01-01, Ending date: 2017-12-31
  * Reference System Information: Reference system identifier: 4326, Code space: EPSG
    * However, when loading in GeoPandas, it says the CRS is 'epsg:3857'
  * Version: 6.14(3.0.1)
* typo in LDC name 'CONNETICUT NATURAL GAS CORP' (missing C)

**Notes on coordinate systems:**
* EPSG:4326: WGS 84 -- WGS84 - World Geodetic System 1984, used in GPS
  * https://epsg.io/4326
  * Projected bounds: -180.0 -90.0 / 180.0 90.0 
* EPSG:3857: WGS 84 / Pseudo-Mercator -- Spherical Mercator, Google Maps, OpenStreetMap, Bing, ArcGIS, ESRI
  * https://epsg.io/3857
  * Projected bounds: -20026376.39 -20048966.10 / 20026376.39 20048966.10

In [ ]:
ornl_path = lca_path + 'ORNL Natural Gas Local Distribution Company Service Territories (from DHS HIFLD) shp/'
ornl_file = 'NG_LDC_Service_Terr.shp'
ornl_init = gpd.read_file(ornl_path + ornl_file)
print(f"initial ORNL CRS: {ornl_init.crs}")

# convert to equal-area projection
# Stack Overflow recommended ESPG 3035: https://gis.stackexchange.com/a/359955
# but that is centered on Europe: https://epsg.io/3035
# Can instead try ESPG 2163 (https://epsg.io/2163); "US National Atlas Equal Area", with UoM: m
ldcs_ornl = ornl_init.to_crs(epsg=2163)
print(f"final ORNL CRS: {ldcs_ornl.crs}")

ldcs_ornl = ldcs_ornl.rename(columns={
    'NAME': 'LDC name', 
    'STATE': 'LDC HQ state?', 
    'AREASQKM': 'LDC area km2'
})
ldcs_ornl['EIA company ID'] = ldcs_ornl['COMPID'] + ldcs_ornl['LDC_STATE']

# filter out those with 'COMPID' == 'NOT AVAILABLE'
ldcs_ornl = ldcs_ornl.loc[ldcs_ornl['COMPID']!='NOT AVAILABLE']

# ldcs_ornl['GPD area km2'] = ldcs_ornl.area / 1e6
# ldcs_ornl['area ratio'] = ldcs_ornl['GPD area km2'].div(ldcs_ornl['LDC area km2'])

ldcs_keep_cols = ['LDC name', 'EIA company ID', 'geometry']  # , 'LDC area km2', 'GPD area km2', 'area ratio'
ldcs_ornl = ldcs_ornl[ldcs_keep_cols]

In [ ]:
def fix_ornl_eia_ids(ldcs_ornl, ldc_name, wrong_id, correct_id):
    df = ldcs_ornl
    
    # get index for row to change
    change_df = df.loc[
        (df['LDC name']==ldc_name) & 
        (df['EIA company ID']==wrong_id)]
    
    if len(change_df)==1:
        change_idx = change_df.index[0]
        df.at[change_idx, 'EIA company ID'] = correct_id
        print(f"changed {ldc_name}")
        
    elif len(change_df)==0:
        print(f"combination of name/ID not found: {ldc_name}, {wrong_id}")
        
    elif len(change_df) > 1:
        print(f"found more than one row:")
        print(change_df)
        
    else:
        print(f"Unexpected other case for {ldc_name}, {wrong_id}")
        
    ldcs_ornl
    
    return ldcs_ornl

In [ ]:
# Fix New York state shapefile issues:

# in ORNL:
# Problem is that National Grid territory shown combines upstate, Brooklyn/Staten Island, and rest of Long Island
# Those are three different EIA IDs: 17610204NY, 17601565NY, 17617823NY
# However, in ORNL listed as simply: 17610204NY

# simplest solution: change ORNL EIA company ID to 17610204NY_17601565NY_17617823NY
# then in EIA data, combine those three National Grid entries into one
# This is not ideal because it's combining upstate NY and NYC, where consumption may be different
# But it's a straightforward way to generate an estimate, and which is similar to the situation in many other urban areas

ldcs_ornl['EIA company ID'] = ldcs_ornl['EIA company ID'].replace(
    '17610204NY', '17610204NY_17601565NY_17617823NY'
)

In [ ]:
# Additional notes:

# Others that are OK in ORNL:

# Manhattan, north of Manhattan, part of Queens:
# comp_full: “Consolidated Edison”; EIA company ID: 17602810NY

# north of Con Ed territory & overlapping with NYC urban area
# comp_full: “Central Hudson Gas and Electric”; EIA company ID: 17602168NY
# comp_full: “NYS Electric Gas”; EIA company ID: 17610199NY
# comp_full: “Orange and Rockland Utilities”; EIA company ID: 17610536NY

# ================
# New York State gas territories shapefile:

# ConEd seems to be wrong; it shows service for Staten Island
# However, ConEd says its Staten Island service is "electric only"
# https://www.coned.com/en/business-partners/service-territories

# Brooklyn (and Staten Island?)
# comp_full: “National Grid - NYC”; EIA company ID: 17601565NY

# rest of Long Island:
# comp_full: “National Grid - Long Island”; EIA company ID: 17617823NY

In [ ]:
# data cleaning

# change EIA company IDs for those that had duplicates, so something was definitely wrong
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'CENTREVILLE, TOWN OF', '17600003MS', '17602193MS')
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'TISHOMINGO NATURAL GAS SYSTEM', '17600003MS', '17619898MS')
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'LOBELVILLE, CITY OF', '17600183TN', '17608530TN')
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'ST. LAWRENCE GAS', '17614659NY', '17611894NY')
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'HALLOCK, CITY OF', '17617865MN', '17605402MN')
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'MIDAMERICAN ENERGY', '17631004NE', '17650920NE')

# Duplicated IDs that were correct:
# BLACK HILLS ENERGY has correct EIA company ID
# VALLEY ENERGY, INC. has correct EIA company ID
# Loretto has correct EIA company ID
# HIBBING PUBLIC UTILITY COMM. has correct EIA company ID

# There is an issue in the data with EIA company ID 17614635SD; 
# These are for Humboldt & Watertown (SD); had issue where these were separate in PHMSA, but didn't have matches in EIA
# Did ORNL data derive from PHMSA, and they created their own crosswalk to EIA???
# Anyway, exclude EIA company ID 17614635SD
ldcs_ornl = ldcs_ornl.loc[ldcs_ornl['EIA company ID']!='17614635SD']

# There is only 1 LDC name that's repeated; also only 1 COMPID repeated
# both are for the same LDC: 'BLUEFIELD GAS', COMPID: 17601305
# res & comm deliveries are the same, but area and geometry aren't the same
# may be that one is serving in WV and one serving in VA (based on 'LDC_STATE')
# guessed that VA one might be ID 17673805VA ("APPALACHIAN NATURAL GAS"), 
# but ORNL set already has an LDC in VA of that name (COMPID 17673805)
# to avoid problems, exclude 'BLUEFIELD GAS', COMPID: 17601305
ldcs_ornl = ldcs_ornl.loc[(ldcs_ornl['EIA company ID']!='17601305WV') & (ldcs_ornl['LDC name']!='BLUEFIELD GAS')]
ldcs_ornl = ldcs_ornl.loc[(ldcs_ornl['EIA company ID']!='17601305VA') & (ldcs_ornl['LDC name']!='BLUEFIELD GAS')]

In [ ]:
# IDs in ORNL not found in my EIA files:
# explanations and changes as needed

# Washington Gas:
# DC portion has same ID as crosswalk
# VA portion uses 17616173VA; that hasn't been used since 2004; change to 17671254VA
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'WASHINGTON GAS', '17616173VA', '17671254VA')
# MD portion uses 17671254MD; I don't see that used anywhere (neither in my files nor online); change to 17616172MD
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'WASHINGTON GAS', '17671254MD', '17616172MD')

# 17606391ID (INTERMOUNTAIN GAS COMPANY) not in ORNL? (What is serving Boise, ID, in ORNL?)
# change INTERMOUNTAIN GAS CO 17606931ID to 17606391ID
# seems there was a typo in manually entering; digits switched
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'INTERMOUNTAIN GAS CO', '17606931ID', '17606391ID')

# 17672060LA (ATMOS ENERGY CORPORATION) not in ORNL? (What is serving New Orleans, LA, in ORNL?)
# change ATMOS ENERGY - LOUSIANA DIVISION 1760016LA to 17672060LA
# seems that the ID is improperly formatted; not enough digits; also typo
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'ATMOS ENERGY - LOUSIANA DIVISION', '1760016LA', '17672060LA')

# 17699051NC (PIEDMONT NATURAL GAS) not in ORNL
# (ORNL map doesn't show any LDC for the core of Charlotte, NC; only service for some outlying parts; is that correct???)
# change PIEDMONT NATURAL GAS 1769905NC to 17699051NC
# seems there was a typo in manually entering; digit missing
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'PIEDMONT NATURAL GAS', '1769905NC', '17699051NC')

# 17675803NH (LIBERTY UTILITIES DBA ENERGY NORTH N) not in ORNL? What is serving Manchester, NH?
# change LIBERTY UTILITIES (ENERGYNORTH) 17604829NH to 17675803NH
# ID used seems to be old one, last used in 2011
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'LIBERTY UTILITIES (ENERGYNORTH)', '17604829NH', '17675803NH')

# 17673850NM (NEW MEXICO GAS COMPANY) not in ORNL? What is serving Albuquerque, NM?
# change NEW MEXICO GAS COMPANY 17617270NM to 17673850NM
# ID used seems to be old one, last used in 2009
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'NEW MEXICO GAS COMPANY', '17617270NM', '17673850NM')

# 17673240WV (MOUNTAINEER GAS CO) not in ORNL? What is serving Charleston, WV?
# That ID hasn't been used since 2004
# change MOUNTAINEER GAS CO. 17616239WV to 17673240WV
ldcs_ornl = fix_ornl_eia_ids(ldcs_ornl, 'MOUNTAINEER GAS CO.', '17616239WV', '17673240WV')

In [ ]:
def subtract_area_from_territory(df, ldc_a_id, ldc_b_id):
    """
    When one LDC (LDC A) reports covering a large area that encompasses territory of another LDC (LDC B),
    but it turns out LDC A doesn't actually cover the territory of LDC B,
    then this function subtracts, from LDC A, the territory of LDC B.
    """
    
    df_no_change = df.loc[df['EIA company ID']!=ldc_a_id]
    ldc_a = df.loc[df['EIA company ID']==ldc_a_id].reset_index(drop=True)
    ldc_b = df.loc[df['EIA company ID']==ldc_b_id].reset_index(drop=True)
    
    # TEST
    if ldc_a.crs['init'] != ldc_b.crs['init']:
        print("ERROR!" + f" Mismatch of CRS: ldc_a.crs: {ldc_a.crs}; ldc_b.crs: {ldc_b.crs}")
    # END OF TEST
    
    ldc_a_diff_ldc_b = gpd.overlay(ldc_a, ldc_b, how='difference').reset_index(drop=True)
    
    # recombine
    df_new = df_no_change.append(ldc_a_diff_ldc_b).reset_index(drop=True)
    
    # test len
    if len(ldcs_ornl)==len(df_new):
        pass
    else:
        print("Error!" + f" There is a mismatch in the lens: len(ldcs_ornl): {len(ldcs_ornl)} & len(df): {len(df)}")
    # end of test len
        
    return df_new

In [ ]:
def reconfigure_washington_gas(ldcs_ornl, state_bound):
    """
    Washington Gas MD includes parts in WV & VA
    Washington Gas VA is only VA--but doesn't include all the operations in VA, because of the part included with MD
    Washington Gas DC is OK, so don't need to bring it in
    """
    
    # dissolve Washington Gas territories stated to be in VA & MD; create one multipolygon
    mask_wash_gas_to_change = ldcs_ornl['EIA company ID'].isin(['17671254VA', '17616172MD'])
    wash_gas_to_change = ldcs_ornl.loc[mask_wash_gas_to_change]
    wash_gas_to_change = wash_gas_to_change.dissolve(by='LDC name')[['geometry']].reset_index()
    
    # change state_bound CRS if need be
    if ldcs_ornl.crs['init']=='epsg:2163':
        state_bound = state_bound.to_crs(epsg=2163)
    else:
        print(f"ldcs_ornl.crs: {ldcs_ornl.crs}; state_bound.crs: {state_bound.crs}")
    
    new_entries_list = [] # initialize
    
    # TEST
    if state_bound.crs['init'] != wash_gas_to_change.crs['init']:
        print("ERROR!" + f" Mismatch of CRS: state_bound_sel.crs: {state_bound_sel.crs}; wash_gas_to_change.crs: {wash_gas_to_change.crs}")
    # END OF TEST
    
    # change geometry for the selected EIA IDs
    # from the dissolved geometry, select only the part within the state listed in the EIA ID
    for eia_id in ['17671254VA', '17616172MD']:
        state_abbrev = eia_id[-2:]
        state_bound_sel = state_bound.loc[state_bound.index==state_abbrev]
        wash_gas_new = gpd.overlay(state_bound_sel, wash_gas_to_change, how='intersection')
        wash_gas_new['EIA company ID'] = eia_id
        
        new_entries_list += [wash_gas_new]

    # concat new entries, then append to main gdf
    new_entries_gdf = pd.concat(new_entries_list, sort=False)
    not_changed = ldcs_ornl.loc[~mask_wash_gas_to_change]
    
    ldcs_ornl = not_changed.append(new_entries_gdf, sort=False).reset_index(drop=True)
    
    return ldcs_ornl

In [ ]:
def reconfigure_northeast_ohio_gas(ldcs_ornl, state_bound):
    """
    Northeast Ohio Natural Gas Corp (17695242OH) took over:
    Orwell Natural Gas Co. (17695266OH)
    Brainard Gas Corp. (17695236OH)
    https://www.neogas.com/PDF/FINAL MERGER BILL INSERT.pdf
    
    Combine their service territories into one.
    """
    
    # dissolve Washington Gas territories stated to be in VA & MD; create one multipolygon
    mask_to_change = ldcs_ornl['EIA company ID'].isin(['17695242OH', '17695266OH', '17695236OH'])
    not_changed = ldcs_ornl.copy().loc[~mask_to_change]
    
    to_change = ldcs_ornl.copy().loc[mask_to_change]
    to_change['LDC name'] = 'NORTHEAST OHIO NATURAL GAS + ORWELL + BRAINARD'
    to_change['EIA company ID'] = '17695242OH_17695266OH_17695236OH'
    to_change = to_change.dissolve(by=['LDC name', 'EIA company ID'])[['geometry']].reset_index()
    
    
    # change state_bound CRS if need be
    if ldcs_ornl.crs['init']=='epsg:2163':
        state_bound = state_bound.to_crs(epsg=2163)
    else:
        print(f"ldcs_ornl.crs: {ldcs_ornl.crs}; state_bound.crs: {state_bound.crs}")
    
    # TEST
    if state_bound.crs['init'] != to_change.crs['init']:
        print("ERROR!" + f" Mismatch of CRS: state_bound_sel.crs: {state_bound_sel.crs}; to_change.crs: {to_change.crs}")
    # END OF TEST
    
    ldcs_ornl = not_changed.append(to_change, sort=False).reset_index(drop=True)
    
    return ldcs_ornl

In [ ]:
ldcs_ornl = reconfigure_washington_gas(ldcs_ornl, state_bound)
ldcs_ornl = reconfigure_northeast_ohio_gas(ldcs_ornl, state_bound)

In [ ]:
# For Omaha, NE: 
# from Black Hills Energy (17631004NE), subtract territory of Metropolitian Utilities of Omaha (17609407NE)
ldcs_ornl = subtract_area_from_territory(ldcs_ornl, '17631004NE', '17609407NE')

# For Chicago, IL:
# from NICOR GAS COMPANY (17610322IL), subtract territory of PEOPLES GAS LT & COKE CO (17610960IL)
ldcs_ornl = subtract_area_from_territory(ldcs_ornl, '17610322IL', '17610960IL')

# For Mesa, AZ:
# from SOUTHWEST GAS CORPORATION (17616576AZ), subtract territory of MESA CITY OF (17609381AZ)
# based on: https://www.selectmesa.com/business-environment/utilities
ldcs_ornl = subtract_area_from_territory(ldcs_ornl, '17616576AZ', '17609381AZ')

In [ ]:
# remove territory from Atlanta Gas Light (17600626GA) where it overlaps with Austell (17600689GA)
ldcs_ornl = subtract_area_from_territory(ldcs_ornl, '17600626GA', '17600689GA')

# note, for Lawrenceville (17608261GA), couldn't see evidence that it has exclusive service in the area
# so not subtracting Lawrenceville territory from Atlanta Gas Light territory

In [ ]:
# Notes: there are some mismatches in names between ORNL and recent EIA data, but they are actually OK

# for EIA company ID 17611093PA:
# EIA has PEOPLES GAS COMPANY, but ORNL has "PHILLIPS T W GAS & OIL CO"
# This is OK; Peoples Gas website says (https://www.peoples-gas.com/about/): 
# "Peoples has been providing natural gas service in western Pennsylvania for more than 130 years. 
# In the last five years, Peoples purchased the T.W. Phillips Gas & Oil Company and Equitable Gas."

# for EIA company ID 17616572MO:
# EIA has "SPIRE MISSOURI INC", but ORNL has "LACLEDE GAS CO"
# Spire took over Laclede: https://www.spireenergy.com/welcome-laclede-gas-customers

In [ ]:
# Note: ORNL includes KINDER MORGAN INC (17617235NE) covering NE
# EIA Company List says that is actually BLACK HILLS GAS DISTRIBUTION LLC
# However, that has gone inactive; was last active 2016
# so exclude 17617235NE from ORNL
# (also fixes problem with Lincoln, NE & Omaha, NE)
ldcs_ornl = ldcs_ornl.loc[ldcs_ornl['EIA company ID']!='17617235NE']

In [ ]:
# Tuscarora (17695180CA) is a transmission company;
# Still active in CA in 2018, but had no res-comm sales, and very little sales to any end-consumers
# exclude from ORNL
ldcs_ornl = ldcs_ornl.loc[ldcs_ornl['EIA company ID']!='17695180CA']

In [ ]:
# TEST: check if there are any duplicated EIA company IDs
dups = ldcs_ornl.loc[ldcs_ornl['EIA company ID'].duplicated(keep=False)].sort_values(by='EIA company ID')
if len(dups)==0:
    pass
else:
    print(dups)

## EIA data on volumes
Merge in EIA data on gas volumes for specified analysis year

In [ ]:
def consolidate_national_grid_in_new_york(df):
    """
    Problem with ORNL data for National Grid in New York: 
    It listed a single EIA company ID, but that was actually only National Grid operations in upstate NY.
    So in ORNL, changed EIA company ID to be composite of all National Grid operations in NY state.
    
    Here, consolidate all National Grid operations in NY state, 
    so that gas volumes from EIA correspond to territory in ORNL.
    """
    # combine National Grid listings for New York State into one row
    nat_grid_ids = ['17610204NY', '17601565NY', '17617823NY']
    nat_grid_nys = df.copy().loc[df['EIA company ID'].isin(nat_grid_ids)]
    nat_grid_nys['EIA company ID'] = '17610204NY_17601565NY_17617823NY'
    nat_grid_nys['Company Name'] = 'NATIONAL GRID (ALL NYS)'
    nat_grid_nys = nat_grid_nys.groupby(['Year', 'State', 'EIA company ID', 'Company Name']).sum().reset_index()
    
    # exclude the individual rows from df
    df = df.loc[~df['EIA company ID'].isin(nat_grid_ids)]
    
    # append the consolidated row
    df = df.append(nat_grid_nys, sort=False).reset_index()
    
    return df

In [ ]:
def consolidate_northeast_ohio_gas(df):
    """
    Problem with LDCs in Ohio: 
    Northeast Ohio Natural Gas Corp (17695242OH) took over:
    Orwell Natural Gas Co. (17695266OH)
    Brainard Gas Corp. (17695236OH)
    https://www.neogas.com/PDF/FINAL MERGER BILL INSERT.pdf
    
    Combine them all into one.
    """
    # combine National Grid listings for New York State into one row
    ne_oh_ids = ['17695242OH', '17695266OH', '17695236OH']
    ne_oh = df.copy().loc[df['EIA company ID'].isin(ne_oh_ids)]
    ne_oh['EIA company ID'] = '17695242OH_17695266OH_17695236OH'
    ne_oh['Company Name'] = 'NORTHEAST OHIO NATURAL GAS + ORWELL + BRAINARD'
    ne_oh = ne_oh.groupby(['Year', 'State', 'EIA company ID', 'Company Name']).sum().reset_index()
    
    # exclude the individual rows from df
    df = df.loc[~df['EIA company ID'].isin(ne_oh)]
    
    # append the consolidated row
    df = df.append(ne_oh, sort=False).reset_index()
    
    return df

## Census tracts

In [ ]:
def compile_census_tract_population_data():
    acs_2018_path = lca_path + 'US Census/population by census tract - ACS 5-Year (2018)/'
    part1_path = 'AL-FL ACSDT5Y2018.B01003_2020-10-30T180506/'
    part1_file = 'ACSDT5Y2018.B01003_data_with_overlays_2020-10-30T180457.csv'

    part2_path = 'GA-ME ACSDT5Y2018.B01003_2020-10-30T180822/'
    part2_file = 'ACSDT5Y2018.B01003_data_with_overlays_2020-10-30T180807.csv'

    part3_path = 'MD-NH ACSDT5Y2018.B01003_2020-10-30T181100/'
    part3_file = 'ACSDT5Y2018.B01003_data_with_overlays_2020-10-30T181052.csv'

    part4_path = 'NJ-RI ACSDT5Y2018.B01003_2020-10-30T181340/'
    part4_file = 'ACSDT5Y2018.B01003_data_with_overlays_2020-10-30T181331.csv'

    part5_path = 'SC-WY ACSDT5Y2018.B01003_2020-10-30T181603/'
    part5_file = 'ACSDT5Y2018.B01003_data_with_overlays_2020-10-30T181553.csv'
    
    part1 = pd.read_csv(acs_2018_path + part1_path + part1_file, header=1)
    part2 = pd.read_csv(acs_2018_path + part2_path + part2_file, header=1)
    part3 = pd.read_csv(acs_2018_path + part3_path + part3_file, header=1)
    part4 = pd.read_csv(acs_2018_path + part4_path + part4_file, header=1)
    part5 = pd.read_csv(acs_2018_path + part5_path + part5_file, header=1)
    
    # concat all into one df
    df = pd.concat([part1, part2, part3, part4, part5], sort=False)
    df = df.drop('Margin of Error!!Total', axis=1)
    df = df.rename(columns={
        'id': 'tract full ID',
        'Estimate!!Total': 'tract pop',
    })
    
    # split out GEOIDs & states
    df['tract GEOID'] = df['tract full ID'].str.split('1400000US').str[-1].astype(str)
    df['state'] = df['Geographic Area Name'].str.split(', ').str[-1]
    df = df.drop('tract full ID', axis=1)
    
    tract_pop = df
    
    return tract_pop

In [ ]:
tract_pop = compile_census_tract_population_data()

### Census tract boundaries

In [ ]:
def read_tract_st(FIPS_code):
    # read census tract shapefile (one file for each state) 
    tract_st_path = lca_path + f'TIGER-Line Census Tract 2018/tl_2018_{FIPS_code}_tract/'
    tract_st_file = f'tl_2018_{FIPS_code}_tract.shp'
    tract_st = gpd.read_file(tract_st_path + tract_st_file)

    tract_keep_cols = ['GEOID', 'geometry'] # 'TRACTCE', 'MTFCC', 'NAME', 'NAMELSAD'
    tract_st = tract_st[tract_keep_cols]
    tract_st = tract_st.rename(columns={'GEOID': 'tract GEOID'})

    tract_st_cart = tract_st.to_crs(epsg=2163)
    
    return tract_st_cart

In [ ]:
def compile_tract_data(fips, contiguous_us_only):
    # iterate through all states (based on FIPS codes) to read files and put into one df
    all_tracts_list = [] # initialize
    for fips_code in fips['STATEFP']:
        print(f"processing FIPS code {fips_code}")
        tract_st_cart = read_tract_st(fips_code)
        all_tracts_list += [tract_st_cart]

    all_tracts = pd.concat(all_tracts_list, sort=False)
    
    # convert to equal area projection, before calculating area
    all_tracts = all_tracts.to_crs(epsg=2163)
    print(f"all_tracts.crs: {all_tracts.crs}")

    # calculate total area of each tract
    all_tracts['tract km2'] = all_tracts.area * km2_per_m2

    # merge population data; attribution merge (not spatial merge)
    # https://geopandas.org/mergingdata.html
    all_tracts = all_tracts.merge(tract_pop, on='tract GEOID')

    all_tracts['state abbrev'] = all_tracts['state'].replace(states_full_to_abbrev_dict)
    
    all_tracts['pop per km2'] = all_tracts['tract pop'] / all_tracts['tract km2']
    
    all_tracts = all_tracts.drop([
        'Geographic Area Name',
        # 'NAMELSAD'
    ], axis=1)
    
    return all_tracts

In [ ]:
# compile the tracts from state files
all_tracts = compile_tract_data(fips, contiguous_us_only)

In [ ]:
# # for graphics
# tract_ga_pop = all_tracts.copy().loc[all_tracts['state']=='Georgia']
# tract_ga_pop['pop per km2'] = tract_ga_pop['tract pop'].div(tract_ga_pop['tract km2'])

# # export
# tract_ga_pop.to_file('tract_ga_pop', driver ='ESRI Shapefile')

In [ ]:
# # for graphics
# tract_oh_pop = all_tracts.copy().loc[all_tracts['state']=='Ohio']
# tract_oh_pop['pop per km2'] = tract_oh_pop['tract pop'].div(tract_oh_pop['tract km2'])

# # export
# tract_oh_pop.to_file('tract_oh_pop', driver ='ESRI Shapefile')

## Urban areas
* US Census, Cartographic Boundary Files, Urban Areas
* https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

In [ ]:
def read_urban_areas(urban_file):
    """
    Read file from US Census for boundaries of urban areas.
    
    Convert to equal area projection to match CRS of other data sets.
    """
    urban_path = lca_path + 'US Census/cb_2018_us_ua10_500k/'
    urban = gpd.read_file(urban_path + urban_file)
    urban = urban.to_crs(epsg=2163)
    
    print(f"After conversion, urban.crs: {urban.crs}")

    urban = urban.rename(columns={
        'NAME10': 'urban name',
        'ALAND10': 'urban area (units?)',
        'GEOID10': 'urban GEOID',
    })

    urban_keep_cols = ['urban name', 'geometry'] # 'urban GEOID'
    urban = urban[urban_keep_cols]

    return urban

In [ ]:
urban = read_urban_areas(urban_file)

## Test: check that population counts make sense
* Do intersection of census tracts and urban area boundaries
* Calculate population within each urban area
* Compare against Census data on urban area populations

In [ ]:
def test_intersection_urban_tract(urban, all_tracts):
    """
    Iterate over each state, getting its tract data, 
    then doing intersection with all urban areas in the state.
    """
    results_list = [] # initialize
       
    for fips_row in fips.index:
        # get one state's tract data
        state_full = fips.at[fips_row, 'STNAME']
        st_tract = all_tracts.loc[all_tracts['state']==state_full]
    
        # get state abbrev corresponding to state_full
        state_abbrev = states_full_to_abbrev_dict[state_full]
        
        print(f"processing state: {state_full}") # for UI
    
        urban_sel = urban.loc[urban['urban name'].str.contains(state_abbrev)]
    
        # TEST
        if st_tract.crs['init'] != urban_sel.crs['init']:
            print("ERROR!" + f" Mismatch of CRS: st_tract.crs: {st_tract.crs}; urban_sel.crs: {urban_sel.crs}")
        # END OF TEST
    
        # do intersection
        gdf = gpd.overlay(
            st_tract,
            urban_sel,  
            how='intersection')

        gdf['tract int km2'] = gdf['geometry'].area * km2_per_m2
        gdf['tract area fract'] = gdf['tract int km2'].div(gdf['tract km2'])

        tract_pop_int_total = gdf[['tract GEOID', 'tract pop']].drop_duplicates()['tract pop'].sum()

        # calculate sums of fractional areas; used to reweight
        tract_fract_sums = gdf.groupby('tract GEOID')['tract area fract'].sum()
        tract_fract_sums = tract_fract_sums.reset_index()
        tract_fract_sums = tract_fract_sums.rename(columns={'tract area fract': 'tract area fract sum'})

        # merge these sums into main gdf
        gdf = gdf.merge(tract_fract_sums, on=['tract GEOID'])
        gdf['tract area fract reweight'] = gdf['tract area fract'].div(gdf['tract area fract sum'])
        gdf['pop tract'] = gdf['tract pop'] * gdf['tract area fract reweight']

        # TEST: check that population sum is correct
        tract_pop_int_sums = gdf.groupby(['tract GEOID', 'tract pop'])[['pop tract']].sum() # removed from groupby: 'NAMELSAD'
        tract_pop_int_total_after_reweight = tract_pop_int_sums['pop tract'].sum()
        pop_diff = tract_pop_int_total - tract_pop_int_total_after_reweight
        if abs(pop_diff) < 1e-3:
            pass
        else:
            print('Error!' + f" There was a difference in the population after reweighting")
            print(f"initial:    {tract_pop_int_total}")
            print(f"reweighted: {tract_pop_int_total_after_reweight}")
        # END OF TEST
        
        # put FIPS code into gdf
        gdf['FIPS'] = fips_row

        # for each urban area, calculate population
        # (taking into account overlaps handled above)
        df = gdf.groupby(['urban name', 'FIPS'])[['pop tract']].sum()
        df = df.reset_index()
        df = df.rename(columns={'pop tract': 'pop urban'})
        df = df.sort_values(by=['pop urban'], ascending=False)
        
        results_list += [df]
        
    urban_population = pd.concat(results_list, sort=False)
        
    return urban_population

In [ ]:
if run_tests == True:
    urban_population = test_intersection_urban_tract(urban, all_tracts)

## get cities in index

In [ ]:
gim_cities = pd.read_excel(inputs_path + gim_cities_file, dtype={'UACE': str, 'GEOID': str})
sel_urban_areas = gim_cities['Census urban area name'].tolist()

# print(f"Show list before excluding any: {sel_urban_areas}")

# exclude urban areas that will be handled separately
sel_urban_areas.remove('Cedar Rapids, IA')
sel_urban_areas.remove('Lawrence, KS')

#### notes on urban areas excluded, to handle separately:
* **Cedar Rapids, IA:** served by MidAmerican Energy, according to company's map; but not shown in ORNL serving the city
* **Lawrence, KS:** ORNL map shows it being served by both Black Hills Energy and Atmos; however, according to Kansas PUC and the city, service is only from Black Hills

## calculate population served by each LDC (across whole territory)

In [ ]:
def intersection_tract_ldc(ldcs_ornl, all_tracts):
    """
    Iterate over each state, getting its tract data, 
    then doing intersection with all LDCs operating in the state.
    """
    results_list = [] # initialize
    
    for fips_row in fips.index:
        # get one state's tract data
        state_full = fips.at[fips_row, 'STNAME']
        st_tract = all_tracts.loc[all_tracts['state']==state_full]

        print(f"processing {state_full}") # for UI
        
        # get state abbrev corresponding to state_full
        state_abbrev = states_full_to_abbrev_dict[state_full]
        ldcs_st = ldcs_ornl.loc[ldcs_ornl['EIA company ID'].str[-2:]==state_abbrev]
    
        # TEST
        if st_tract.crs['init'] != ldcs_st.crs['init']:
            print("ERROR!" + f" Mismatch of CRS: st_tract.crs: {st_tract.crs}; ldcs_st.crs: {ldcs_st.crs}")
        # END OF TEST
    
        # do intersection
        gdf = gpd.overlay(
            st_tract,
            ldcs_st,  
            how='intersection')

        gdf['tract int km2'] = gdf['geometry'].area * km2_per_m2
        gdf['tract area fract'] = gdf['tract int km2'].div(gdf['tract km2'])

        tract_pop_int_total = gdf[['tract GEOID', 'tract pop']].drop_duplicates()['tract pop'].sum()

        # calculate sums of fractional areas; used to reweight
        tract_fract_sums = gdf.groupby('tract GEOID')['tract area fract'].sum()
        tract_fract_sums = tract_fract_sums.reset_index()
        tract_fract_sums = tract_fract_sums.rename(columns={'tract area fract': 'tract area fract sum'})

        # merge these sums into main gdf
        gdf = gdf.merge(tract_fract_sums, on=['tract GEOID'])
        gdf['tract area fract reweight'] = gdf['tract area fract'].div(gdf['tract area fract sum'])
        gdf['LDC pop tract'] = gdf['tract pop'] * gdf['tract area fract reweight']

        # TEST: check that population sum is correct
        tract_pop_int_sums = gdf.groupby(['tract GEOID', 'tract pop'])[['LDC pop tract']].sum()
        tract_pop_int_total_after_reweight = tract_pop_int_sums['LDC pop tract'].sum()
        pop_diff = tract_pop_int_total - tract_pop_int_total_after_reweight
        if abs(pop_diff) < 1e-3:
            pass
        else:
            print('Error!' + f" There was a difference in the population after reweighting")
            print(f"initial:    {tract_pop_int_total}")
            print(f"reweighted: {tract_pop_int_total_after_reweight}")
        # END OF TEST

        # for each LDC, calculate population served
        df = gdf.groupby(['LDC name', 'EIA company ID'])[['LDC pop tract']].sum()
        
        df = df.reset_index()
        df = df.rename(columns={'LDC pop tract': 'LDC pop served'})
        df = df.sort_values(by=['LDC pop served'], ascending=False)

        results_list += [df]
        
    ldc_population_served = pd.concat(results_list, sort=False)
        
    return ldc_population_served

In [ ]:
ldc_population_served = intersection_tract_ldc(ldcs_ornl, all_tracts)

## In each urban area, for each LDC, calculate population served & gas delivered

In [ ]:
def allocate_tract_pop_to_one_ldc(tract_sel, ldcs_sel_urb_tract, ldcs_sel_urb):
    tract_results = pd.DataFrame() # initialize
    
    # TEST:
    if len(ldcs_sel_urb_tract) > 1:
        print("ERROR!" + f" For this function, supposed to be only 1 LDC; len(ldcs_sel_urb_tract): {len(ldcs_sel_urb_tract)}")
    
    ldcs_sel_urb_tract['pop served'] = ldcs_sel_urb_tract['pop per km2'] * ldcs_sel_urb_tract['km2 served']

    tract_results = tract_results.append(ldcs_sel_urb_tract, sort=False)
    
    # to get portion of tract that is not in LDC's territory:
    no_ldc = gpd.overlay(tract_sel, ldcs_sel_urb, how='difference').reset_index(drop=True)
    no_ldc['km2 served'] = no_ldc.area * km2_per_m2
    no_ldc['pop served'] = no_ldc['pop per km2'] * no_ldc['km2 served']
    no_ldc['urban name'] = sel_urb_name
    no_ldc['EIA company ID'] = 'no LDC and/or not in urban'
    no_ldc['LDC name'] = 'no LDC and/or not in urban'
    tract_results = tract_results.append(no_ldc, sort=False)
    
    # TEST: check that the LDC area & non-LDC area is summing to total area
    tract_results_area_ldc_and_non = tract_results['km2 served'].sum()
    tract_area_tot = (tract_sel.area * km2_per_m2).max()
    diff = tract_area_tot - tract_results_area_ldc_and_non
    fract_diff = diff / tract_area_tot
    if abs(fract_diff) > 0.01:
        print("------ ERROR!" + f" The sum of areas for LDCs and no LDC service was not the same as the tract total area!")
        print(f"Tract total area: {tract_area_tot}; fract_diff: {fract_diff}")

    return tract_results

In [ ]:
def allocate_tract_pop_between_two_ldcs(tract_sel, sel_urban_area, ldcs_ids):
    tract_results = pd.DataFrame() # initialize
    
    ldc1_id = ldcs_ids[0]
    ldc2_id = ldcs_ids[1]

    # split pop between the 2 LDCs, using all 4 combinations:
    # only LDC1, only LDC2, both LDCs, neither LDC

    # get the two LDCs' territories
    ldc1 = ldcs_ornl.loc[ldcs_ornl['EIA company ID']==ldc1_id].reset_index(drop=True)
    ldc2 = ldcs_ornl.loc[ldcs_ornl['EIA company ID']==ldc2_id].reset_index(drop=True)

    # find the two LDCs' territories that are within the selected urban area
    ldc1_urb = gpd.overlay(ldc1, sel_urban_area, how='intersection')
    ldc2_urb = gpd.overlay(ldc2, sel_urban_area, how='intersection')

    # find the two LDCs' territories within the selected urban area that are also within the selected tract
    ldc1_urb_tract = gpd.overlay(ldc1_urb, tract_sel, how='intersection')
    ldc2_urb_tract = gpd.overlay(ldc2_urb, tract_sel, how='intersection')

    try:
        both_ldcs = gpd.overlay(
            ldc1_urb_tract.drop(['LDC name', 'EIA company ID'], axis=1), 
            ldc2_urb_tract[['geometry']], 
            how='intersection')

        if len(both_ldcs) == 0:
            # There were 2 LDCs in the tract, but no overlap in their service territories
            # No overlap requiring split of population, so skip both_ldcs steps
            pass

        elif len(both_ldcs) > 1:
            # There were 2 LDCs in the tract, with at least partially overlapping territories
            # Sort them out
            both_ldcs = both_ldcs.reset_index(drop=True)

            # calculate for portion served by both LDCs
            both_ldcs['km2 served'] = both_ldcs.area * km2_per_m2
            both_ldcs['pop served 2 LDCs'] = both_ldcs['pop per km2'] * both_ldcs['km2 served']
            split_pop = both_ldcs.at[0, 'pop served 2 LDCs']/2
            ldc1_split_pop = both_ldcs.copy()

            for ldc_id in [ldc1_id, ldc2_id]:
                ldc_split = both_ldcs.copy()
                ldc_split['EIA company ID'] = ldc_id
                ldc_split['LDC name'] = ldcs_ornl.loc[ldcs_ornl['EIA company ID']==ldc_id].reset_index(drop=True).at[0, 'LDC name']
                ldc_split['pop served'] = split_pop
                ldc_split = ldc_split.drop('pop served 2 LDCs', axis=1)
                tract_results = tract_results.append(ldc_split, sort=False)

        only_ldc1 = gpd.overlay(ldc1_urb_tract, ldc2_urb_tract, how='difference')
        only_ldc1['km2 served'] = only_ldc1.area * km2_per_m2
        only_ldc1['pop served'] = only_ldc1['pop per km2'] * only_ldc1['km2 served']
        tract_results = tract_results.append(only_ldc1, sort=False)

        only_ldc2 = gpd.overlay(ldc2_urb_tract, ldc1_urb_tract, how='difference')
        only_ldc2['km2 served'] = only_ldc2.area * km2_per_m2
        only_ldc2['pop served'] = only_ldc2['pop per km2'] * only_ldc2['km2 served']
        tract_results = tract_results.append(only_ldc2, sort=False)

        # to get portion of tract that is in neither LDC territory:
        # unitary union or dissolve of two (or more) LDC territories
        # overlay difference, with tract first and territories second
        both_ldcs_ornl = ldcs_ornl.loc[ldcs_ornl['EIA company ID'].isin([ldc1_id, ldc2_id])]
        no_ldc = gpd.overlay(tract_sel, both_ldcs_ornl, how='difference').reset_index(drop=True)
        no_ldc['km2 served'] = no_ldc.area * km2_per_m2
        no_ldc['pop served'] = no_ldc['pop per km2'] * no_ldc['km2 served']
        no_ldc['urban name'] = sel_urb_name
        no_ldc['EIA company ID'] = 'no LDC and/or not in urban'
        no_ldc['LDC name'] = 'no LDC and/or not in urban'
        tract_results = tract_results.append(no_ldc, sort=False)

    except:
        sel_geoid = tract_sel.at[0, 'tract GEOID']
        print(f"both_ldcs intersection failed for tract GEOID: {sel_geoid}, for EIA company IDs: {ldcs_ids}")

    return tract_results

In [ ]:
def allocate_tract_pop_between_major_ldcs_equally(ldcs_sel_urb_tract_majors):
    num_major_ldcs = len(ldcs_sel_urb_tract_majors)
    tract_pop = ldcs_sel_urb_tract_majors['tract pop'].max()
    pop_per_ldc = tract_pop / num_major_ldcs
    ldcs_sel_urb_tract_majors['pop served'] = pop_per_ldc
    
    tract_results = ldcs_sel_urb_tract_majors.drop(['fract cov'], axis=1)
    
    return tract_results

In [ ]:
def allocate_tract_pop_between_top_3_ldcs_equally(top_3):
    num_major_ldcs = len(top_3)
    tract_pop_served = top_3[['tract pop', 'fract cov']].product(axis=1, skipna=None).sum()
    pop_per_ldc = tract_pop_served / num_major_ldcs
    top_3['pop served'] = pop_per_ldc
    
    tract_results = top_3
    
    return tract_results

In [ ]:
# iterate over all cities in Gas Index
all_urb_tract_results = pd.DataFrame() # initialize

for sel_urb_name in sel_urban_areas:
    sel_urban_tract_results_list = [] # initialize
    sel_urban_area = urban.loc[urban['urban name']==sel_urb_name]

    sel_states = sel_urb_name.split(', ')[-1].split('--')
    # print(sel_states) # for db
    tracts_sel_states = all_tracts.loc[all_tracts['state abbrev'].isin(sel_states)]
    # print(len(tracts_sel_states)) # for db
    sel_urban_area_tracts = gpd.overlay(sel_urban_area, tracts_sel_states, how='intersection')
    
    sel_urban_area_tract_geoids_list = sel_urban_area_tracts['tract GEOID'].unique().tolist()
    
    ldcs_sel = ldcs_ornl.loc[ldcs_ornl['EIA company ID'].str[-2:].isin(sel_states)]
    ldcs_sel_urb = gpd.overlay(ldcs_sel, sel_urban_area, how='intersection')
    
    # select LDCs in sel_states
    number_of_tracts = len(sel_urban_area_tract_geoids_list)
    print(f"{sel_urb_name}: processing {number_of_tracts} tracts")

    counter = 0 # initialize
    for sel_geoid in sel_urban_area_tract_geoids_list:
        try:
            # select one tract
            tract_sel = all_tracts.copy().loc[all_tracts['tract GEOID']==sel_geoid].reset_index(drop=True)

            # find the LDCs serving the given tract
            ldcs_sel_urb_tract = gpd.overlay(ldcs_sel_urb, tract_sel, how='intersection')

            ldcs_sel_urb_tract['km2 served'] = ldcs_sel_urb_tract.area * km2_per_m2

            # filter out minor pieces (< 1% of area); if that filters down to 2 or less LDCs, use those
            ldcs_sel_urb_tract['fract cov'] = ldcs_sel_urb_tract['km2 served'] / ldcs_sel_urb_tract['tract km2']
            ldcs_sel_urb_tract = ldcs_sel_urb_tract.copy().loc[ldcs_sel_urb_tract['fract cov'] > 0.01]

            if len(ldcs_sel_urb_tract)==0:
                # no service
                pass

            elif len(ldcs_sel_urb_tract)==1:
                # ldc_id = ldcs_sel_urb_tract['EIA company ID'].tolist()[0]
                # tract_results = allocate_tract_pop_to_one_ldc(tract_sel, ldcs_sel_urb, ldc_id) # sel_urban_area
                # tract_results = allocate_tract_pop_to_one_ldc(tract_sel, ldcs_sel_urb_tract) # sel_urban_area
                tract_results = allocate_tract_pop_to_one_ldc(tract_sel, ldcs_sel_urb_tract, ldcs_sel_urb)
                sel_urban_tract_results_list.append(tract_results)

            elif len(ldcs_sel_urb_tract)==2:
                ldcs_ids = ldcs_sel_urb_tract['EIA company ID'].tolist()
                tract_results = allocate_tract_pop_between_two_ldcs(tract_sel, sel_urban_area, ldcs_ids)
                sel_urban_tract_results_list.append(tract_results)

            elif len(ldcs_sel_urb_tract) > 2:
                # ldcs_sel_urb_tract['km2 served'] = ldcs_sel_urb_tract.area * km2_per_m2
                ldcs_sel_urb_tract = ldcs_sel_urb_tract.sort_values(by=['km2 served'], ascending=False).reset_index(drop=True)

                ldcs_sel_urb_tract_majors = ldcs_sel_urb_tract.copy().loc[ldcs_sel_urb_tract['fract cov']>=0.75]
                if len(ldcs_sel_urb_tract_majors) > 2:
                    # there are more than 2 LDCs that each cover at least 75% of the tract
                    # split the population equally between all the major LDCs
                    tract_results = allocate_tract_pop_between_major_ldcs_equally(ldcs_sel_urb_tract_majors)
                    sel_urban_tract_results_list.append(tract_results)

                else:
                    # not more than 2 LDCs covering >= 75% of area
                    # print(f"case with > 2 LDCs serving tract, but not more than 2 covering >= 75% of area: tract GEOID: {sel_geoid}") # for db

                    # try filtering out small pieces (< 5%); if that filters down to 2 or less LDCs, use those
                    ldcs_sel_urb_tract_excl_minor = ldcs_sel_urb_tract.copy().loc[ldcs_sel_urb_tract['fract cov'] > 0.05]
                    if len(ldcs_sel_urb_tract_excl_minor)==1:
                        print("---- after filtering out < 5%, only 1 LDC")
                        tract_results = allocate_tract_pop_to_one_ldc(tract_sel, ldcs_sel_urb_tract_excl_minor, ldcs_sel_urb)
                        sel_urban_tract_results_list.append(tract_results)

                    elif len(ldcs_sel_urb_tract_excl_minor) == 2:
                        print("---- after filtering out < 5%, only 2 LDCs")
                        ldcs_ids = ldcs_sel_urb_tract_excl_minor['EIA company ID'].tolist()
                        tract_results = allocate_tract_pop_between_two_ldcs(tract_sel, sel_urban_area, ldcs_ids)
                        sel_urban_tract_results_list.append(tract_results)

                    else:
                        # check if top 3 cover about the same fraction; if so, split equally
                        top_3 = ldcs_sel_urb_tract_excl_minor.loc[ldcs_sel_urb_tract_excl_minor.index.isin([0, 1, 2])]

                        if (top_3['fract cov'].std() / top_3['fract cov'].mean()) < 0.2:
                            # the top 3 LDCs cover about the same area; divide area between them equally
                            tract_results = allocate_tract_pop_between_top_3_ldcs_equally(top_3)
                            sel_urban_tract_results_list.append(tract_results)

                        else:
                            # variation among the top 3; choose the 2 largest                    
                            ldcs_sel_urb_tract_excl_minor_largest = ldcs_sel_urb_tract_excl_minor.copy().loc[
                                ldcs_sel_urb_tract_excl_minor.index.isin([0, 1])]

                            # TEST: after filtering step above, make sure only 2 rows remaining
                            if len(ldcs_sel_urb_tract_excl_minor_largest)!=2:
                                print(f"len(ldcs_sel_urb_tract) supposed to be 2, but it is: {len(ldcs_sel_urb_tract_excl_minor_largest)}")
                            # END OF TEST

                            ldcs_ids = ldcs_sel_urb_tract_excl_minor_largest['EIA company ID'].tolist()
                            tract_results = allocate_tract_pop_between_two_ldcs(tract_sel, sel_urban_area, ldcs_ids)
                            sel_urban_tract_results_list.append(tract_results)

            else:
                print("ERROR!" + f" Unexpected value for len(ldcs_sel_urb_tract): {len(ldcs_sel_urb_tract)}")

            counter += 1
            if counter % 100 == 0:
                print(f"-- Processed {counter} tracts; len(sel_urban_tract_results_list): {len(sel_urban_tract_results_list)}")
                
        except: # with try just after for sel_geoid in ...
            print(f"---- Not able to process sel_geoid {sel_geoid}; could be TopologyException")
            print("For tract 28121020302 in Jackson, MS, got error: 'TopologyException: no outgoing dirEdge found at 928240.987367964 -1358326.433380282'")

    # end of one urban area; compile into df & append to main df
    sel_urban_tract_results = pd.concat(sel_urban_tract_results_list, sort=False)
    all_urb_tract_results = all_urb_tract_results.append(sel_urban_tract_results, sort=False)

In [ ]:
all_urb_tract_results = all_urb_tract_results.reset_index()
print(len(all_urb_tract_results))

In [ ]:
# export results
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
all_urb_tract_results.to_excel(lca_path + f'GIM LDCs alloc all_urb_tract_results {save_timestamp}.xlsx')

In [ ]:
# there were 34 tracts with fract served > 1
# renormalize these by adjusting pop served
df = all_urb_tract_results.groupby(
    ['urban name', 'tract GEOID', 'tract pop'])[['pop served']].sum().reset_index()
df['fract served'] = df['pop served'] / df['tract pop']
overshoot = df.loc[df['fract served']>1.00001]
overshoot = overshoot.rename(columns={'fract served': 'overshoot fract served'})

# merge overshoot
df2 = pd.merge(
    all_urb_tract_results, overshoot[['tract GEOID', 'overshoot fract served']],
    on='tract GEOID', how='left')
df2['overshoot fract served'] = df2['overshoot fract served'].fillna(1)
df2['pop served'] = df2['pop served'] / df2['overshoot fract served']
df2['check ratios'] = df2['pop served'] / df2['tract pop']

all_urb_tract_results = df2

In [ ]:
tract_results_ldcs = all_urb_tract_results.loc[all_urb_tract_results['EIA company ID']!='no LDC and/or not in urban']
urb_ldcs_pop_served = tract_results_ldcs.groupby(['urban name', 'EIA company ID'])[['pop served']].sum().reset_index()
print(len(urb_ldcs_pop_served))

# remove all rows with pop served < 1
urb_ldcs_pop_served = urb_ldcs_pop_served.loc[urb_ldcs_pop_served['pop served']>1]
print(len(urb_ldcs_pop_served))

In [ ]:
urb_ldcs_pop_served = urb_ldcs_pop_served.sort_values(by=['urban name', 'pop served'], ascending=[True, False]).reset_index(drop=True)
urb_ldcs_pop_served = urb_ldcs_pop_served.rename(columns={'pop served': '1LDC pop served in 1urb'})

In [ ]:
urb_ldcs_pop_served

## put back in individual cities with special handling

In [ ]:
def fill_in_individual_cities(
    urban_name, eia_id, urb_ldcs_pop_served, urban, all_tracts): # ldc_name

    # ---------------
    # calculate urban population
    # get urban area shape
    urb_shp = urban.loc[urban['urban name']==urban_name]

    # select census tracts for relevant states
    states_covered_abbrev = urban_name.split(', ')[-1].split('--')
    tracts_sel = all_tracts.loc[all_tracts['state abbrev'].isin(states_covered_abbrev)]

    test_for_crs_mismatch(urb_shp, tracts_sel)

    # intersection with census tracts (with population) for states covered: whole urban area
    urb_tract = gpd.overlay(urb_shp, tracts_sel, how='intersection')
    urb_pop = urb_tract['tract pop'].sum()

    # put values into urb_ldcs_pop_served
    df = urb_ldcs_pop_served.copy()
    new_index = urb_ldcs_pop_served.index.max()+1
    df.at[new_index, 'urban name'] = urban_name
    df.at[new_index, 'EIA company ID'] = eia_id
    df.at[new_index, '1LDC pop served in 1urb'] = urb_pop

    urb_ldcs_pop_served_mod = df
    return urb_ldcs_pop_served_mod

In [ ]:
urb_ldcs_pop_served_mod = urb_ldcs_pop_served.copy()

urb_ldcs_pop_served_mod = fill_in_individual_cities(
    'Lawrence, KS', '17681028KS', 
    urb_ldcs_pop_served_mod, urban, all_tracts)

urb_ldcs_pop_served_mod = fill_in_individual_cities(
    'Cedar Rapids, IA', '17650918IA',
    urb_ldcs_pop_served_mod, urban, all_tracts)

## import EIA data on gas deliveries by each LDC

In [ ]:
def read_operations_file_and_calculate_sectoral_groups(operations_year_file, contiguous_us_only):
    """
    Note: for at least one utility (Midwest Energy, in Kansas), there is a mismatch
    between EIA data in operations file and in deliveries file.
    
    In deliveries file, this utility's data is split between two different IDs,
    whereas in operations file, it's under one ID.
    
    The deliveries file is not used in GIM.
    """
    # use EIA-176 "Type of Operation" data set that has customer numbers
    df = pd.read_excel(eia_176_path + operations_year_file, sheet_name='Sheet 1', header=1)
    df = df.rename(columns={
        'Company': 'EIA company ID',
        'Company Name': 'EIA company name',
        'Residential Total Volume (Mcf)': 'LDC total res Mcf', 
        'Commercial Total Volume (Mcf)': 'LDC total comm Mcf',
        'Electric Total Volume (Mcf)': 'LDC total elec Mcf',
        'Industrial Total Volume (Mcf)': 'LDC total ind Mcf',
        'Vehicle Fuel Total Volume (Mcf)': 'LDC total veh Mcf',
    })
    
    df['LDC total all sectors Mcf'] = df[[
        'LDC total res Mcf',
        'LDC total comm Mcf',
        'LDC total elec Mcf',
        'LDC total ind Mcf',
        'LDC total veh Mcf',
    ]].sum(axis=1)
    
    print(f"all US deliveries (Bcf): {round(df['LDC total all sectors Mcf'].sum() / 1e6, 1)}")
    
    df = consolidate_national_grid_in_new_york(df)
    df = consolidate_northeast_ohio_gas(df)
    
    # filter for only contiguous US (depending on parameter setting)
    if contiguous_us_only == True:
        df = df.loc[~df['State'].isin(['AK', 'HI', 'PR'])]
        
    print(f"contiguous US deliveries (Bcf): {round(df['LDC total all sectors Mcf'].sum() / 1e6, 1)}")
        
    # remove all adjustment companies 
    # (none of them have any volumes or customer numbers anyway, at least for 2018)
    df = df.loc[df['EIA company name']!='ADJUSTMENT COMPANY']
    
    print(f"contiguous US deliveries excl. adjustment companies (Bcf): {round(df['LDC total all sectors Mcf'].sum() / 1e6, 1)}")
   
    df = df[[
        'EIA company ID', 'EIA company name', 
        'LDC total res Mcf', 'LDC total comm Mcf', 
        'LDC total elec Mcf', 'LDC total ind Mcf',
        'LDC total veh Mcf',
        'LDC total all sectors Mcf']]
    
    operations_year = df
    
    return operations_year

In [ ]:
operations_year = read_operations_file_and_calculate_sectoral_groups(
    operations_year_file, contiguous_us_only)

# Merge city & LDC data: 
* bring in ldc_population_served, with total population served by each LDC
* calculate the share of each LDC's population served that is within each urban area
* then merge in EIA data on gas deliveries
* assume gas deliveries are proportional to population, to calculate gas deliveries by each LDC to each urban area

In [ ]:
def create_all_ldc_urban(urb_ldcs_pop_served_mod, ldc_population_served, operations_year):
    df = pd.merge(urb_ldcs_pop_served_mod, ldc_population_served, on='EIA company ID', how='left')
    df['LDC fract to 1urb'] = df['1LDC pop served in 1urb'] / df['LDC pop served']

    df = pd.merge(df, operations_year, on='EIA company ID', how='left')
    df['1LDC 1urb res Mcf'] = df['LDC total res Mcf'] * df['LDC fract to 1urb']
    df['1LDC 1urb comm Mcf'] = df['LDC total comm Mcf'] * df['LDC fract to 1urb']
    df['1LDC 1urb res-comm Mcf'] = df[['1LDC 1urb res Mcf', '1LDC 1urb comm Mcf']].sum(axis=1)
    df = df.drop(['1LDC 1urb res Mcf', '1LDC 1urb comm Mcf'], axis=1)
    
    df['1LDC 1urb elec Mcf'] = df['LDC total elec Mcf'] * df['LDC fract to 1urb']
    df['1LDC 1urb ind Mcf'] = df['LDC total ind Mcf'] * df['LDC fract to 1urb']
    # df['1LDC 1urb veh Mcf'] = df['LDC total veh Mcf'] * df['LDC fract to 1urb']
    df['1LDC 1urb all sect Mcf'] = df['LDC total all sectors Mcf'] * df['LDC fract to 1urb']

    df = df.drop([
        'LDC name',
        '1LDC pop served in 1urb', 'LDC pop served',
        'LDC total res Mcf', 'LDC total comm Mcf', 'LDC total elec Mcf',
        'LDC total ind Mcf', 'LDC total veh Mcf', 
        'LDC total all sectors Mcf',
    ], axis=1)

    all_ldc_urban = df
    return all_ldc_urban

In [ ]:
all_ldc_urban = create_all_ldc_urban(urb_ldcs_pop_served_mod, ldc_population_served, operations_year)

## calculate quantities of methane leakage from each LDC allocated to each urban area

In [ ]:
def read_ldc_leak_rcei():
    # read results from previous notebook (GIM LDC leakage)
    ldc_leak_rcei_file = f'GIM LDC res-comm-elec-ind leak for {data_year} {ldc_leak_rcei_timestamp}.csv'
    ldc_leak = pd.read_csv(lca_path + ldc_leak_rcei_file)

    ldc_leak_keep_cols = [
        'EIA company ID',
        # 'total deliv NG Mcf',
        'dist pipe res-comm leak CH4 Gg', 'res-comm meter leak CH4 Gg', 'behind-the-meter res-comm CH4 Gg',
        # 'LDC res-comm leak total CH4 Gg',
        'dist pipe elec leak CH4 Gg', 'elec meter leak CH4 Gg',
        # 'LDC elec leak total CH4 Gg', 
        'dist pipe ind leak CH4 Gg', 'indust meter leak CH4 Gg',
        # 'LDC ind leak total CH4 Gg'
    ]
    ldc_leak = ldc_leak[ldc_leak_keep_cols]
    ldc_leak = ldc_leak.set_index('EIA company ID')
    
    return ldc_leak

In [ ]:
def calculate_leakage_rates_by_urban_area(all_ldc_urban, ldc_leak, operations_year):
    # merge in leakage data from previous notebook
    df = pd.merge(all_ldc_urban, ldc_leak, on='EIA company ID', how='left')
    leak_cols = [
        'dist pipe res-comm leak CH4 Gg', 'res-comm meter leak CH4 Gg', 'behind-the-meter res-comm CH4 Gg',
        'dist pipe elec leak CH4 Gg', 'elec meter leak CH4 Gg',
        'dist pipe ind leak CH4 Gg', 'indust meter leak CH4 Gg'
    ]
    leak_cols_new = [] # initialize
    for col in leak_cols:
        new_col = col + ' 1urb'
        df[new_col] = df[col] * df['LDC fract to 1urb']   
        df = df.drop(col, axis=1)
        leak_cols_new += [new_col]

    ungrouped = df.copy()
        
    all_ldc_urban_vol_cols = [
        '1LDC 1urb res-comm Mcf', 
        '1LDC 1urb elec Mcf', 
        '1LDC 1urb ind Mcf', '1LDC 1urb all sect Mcf'
    ]
        
    df = df.groupby('urban name')[leak_cols_new + all_ldc_urban_vol_cols].sum()
    df = df.reset_index()
    for col in df.columns:
        if '1LDC 1urb' in col:
            new_col = col.replace('1LDC 1urb', '1urb')
            df = df.rename(columns={col: new_col})

    # leak rates:
    # calculate leakage rate for each component, based on the gas deliveries for the corresponding sector
    for col in leak_cols_new:
        if 'res-comm' in col:
            sector = 'res-comm'
        elif 'elec' in col:
            sector = 'elec'
        elif 'ind' in col:
            sector = 'ind'
#         elif 'veh' in col:
#             sector = 'veh'
        else:
            print("ERROR!" + f" Unexpected value for col: {col}")

        rate_col = col.replace('CH4 Gg', 'CH4 g/Mcf')
            
        df[rate_col] = (df[col] * g_per_gg) / df[f'1urb {sector} Mcf']
        
    result = df
    return(result, ungrouped)
# end of calculate_leakage_rates_by_urban_area

In [ ]:
ldc_leak = read_ldc_leak_rcei()
(result, ungrouped) = calculate_leakage_rates_by_urban_area(all_ldc_urban, ldc_leak, operations_year)

In [ ]:
# export
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
ungrouped.to_excel(
    lca_path + f'GIM results - LDCs allocation to cities - ungrouped {save_timestamp}.xlsx', 
    index=False)

In [ ]:
ungrouped.loc[ungrouped['urban name']=='Boise City, ID'].T

In [ ]:
# Bcf
ungrouped.loc[ungrouped['urban name']=='Indianapolis, IN']['1LDC 1urb all sect Mcf'].sum()/1e6

## compile results for cities in index

In [ ]:
# Production area leakage results, by consuming state:
prod_leak_file = f'GIM production area leakage rate by consuming state for 2018 {prod_leak_timestamp}.csv'
prod_leak = pd.read_csv(lca_path + prod_leak_file)
if prod_leak.columns[0]=='Unnamed: 0':
    prod_leak = prod_leak.rename(columns={'Unnamed: 0': 'state'})
prod_leak['state abbrev'] = prod_leak['state'].replace(states_full_to_abbrev_dict)
prod_leak = prod_leak.loc[~prod_leak['state'].isin(['Canada', 'Mexico'])]

# simple approach: assign a single state to each urban area
# (more complex approach would be to assign a weighted average across states, when more than one state)
# (however, there generally isn't a big difference in the production-area leakage value for neighboring consuming states,
# at least for any urban areas in the index that cross state boundaries)
result['state abbrev'] = result['urban name'].str.split(', ').str[-1].str.split('--').str[0]

result_states_initial = result['state abbrev'].unique().tolist()
result = pd.merge(
    result,
    prod_leak[['state abbrev', 'prod leak g CH4/Mcf dry gas']],
    on='state abbrev', how='outer'
)
for name in result_states_initial:
    if name not in result['state abbrev'].unique().tolist():
        print("ERROR!" + f" Change in urban name: {name}")

In [ ]:
# Transmission area leakage results
trans_leak_file = f'GIM trans leak by city for 2018 {trans_leak_timestamp}.csv'
trans_leak = pd.read_csv(lca_path + trans_leak_file)

result_urban_names_initial = result['urban name'].unique().tolist()
result = pd.merge(
    result, 
    trans_leak[['urban name', 'trans leak g CH4/Mcf']].dropna(subset=['urban name']), 
    on='urban name', how='outer')
for name in result_urban_names_initial:
    if name not in result['urban name'].unique().tolist():
        print("ERROR!" + f" Change in urban name: {name}")

In [ ]:
# show rows with nan; if only DE & NJ, that's expected
result.loc[result['urban name'].isna()]

In [ ]:
result = result.loc[result['urban name'].isna()==False]
result = result.drop('state abbrev', axis=1)

## City leakage measurements
Calculate additional emissions based on measurements of leakage within cities

In [ ]:
df = pd.read_excel(inputs_path + parameters_file, sheet_name='urban area leakage')
citywide_meas_leak_fracts = df.set_index('urban area')['leakage rate (% of CH4 delivered that leaks)']

In [ ]:
# calculate the corresponding methane leakage rate in g CH4/Mcf gas delivered
citywide_leak_g_mcf = citywide_meas_leak_fracts * conversion_consumer_ng_mcf_to_ch4_gg * g_per_gg
citywide_leak_g_mcf.name = 'citywide measured g CH4/Mcf'

# merge with results
result = pd.merge(
    result, citywide_leak_g_mcf, 
    left_on='urban name', right_index=True, 
    how='outer',
)

In [ ]:
# calculate the corresponding quantity of methane leakage (Gg/y)
# based on GIM estimate of NG deliveries to each urban area
result['meas corr leak CH4 Gg/y'] = result[
    ['citywide measured g CH4/Mcf', '1urb all sect Mcf']].product(axis=1, skipna=False) * gg_per_g

# calculate additional leakage implied by measured citywide leakage
leak_gg_components = [
    'dist pipe res-comm leak CH4 Gg 1urb',
    'res-comm meter leak CH4 Gg 1urb',
    'behind-the-meter res-comm CH4 Gg 1urb',
    'dist pipe elec leak CH4 Gg 1urb', 'elec meter leak CH4 Gg 1urb',
    'dist pipe ind leak CH4 Gg 1urb', 'indust meter leak CH4 Gg 1urb',
]
result['GIM city leak Gg/y'] = result[leak_gg_components].sum(axis=1)
result['add leak CH4 Gg/y'] = result['meas corr leak CH4 Gg/y'] - result['GIM city leak Gg/y']

for row in result.index:
    add_leak = result.at[row, 'add leak CH4 Gg/y']
    if add_leak < 0:
        # replace with zero
        result.at[row, 'add leak CH4 Gg/y'] = 0
        
# partition additional leakage (Gg/y) across sectors & calculate leakage rate
result['add leak res-comm CH4 g/Mcf 1urb rate'] = (result['add leak CH4 Gg/y'] * g_per_gg) / result['1urb all sect Mcf']
result['add leak res-comm CH4 g/Mcf 1urb rate'] = result['add leak res-comm CH4 g/Mcf 1urb rate'].fillna(0)

## Prepare results & export

In [ ]:
result_rescomm = result.copy()[[
    'urban name', 
    'prod leak g CH4/Mcf dry gas', 
    'trans leak g CH4/Mcf',
    'dist pipe res-comm leak CH4 g/Mcf 1urb',
    'res-comm meter leak CH4 g/Mcf 1urb',
    'behind-the-meter res-comm CH4 g/Mcf 1urb',
    'add leak res-comm CH4 g/Mcf 1urb rate'
]]

In [ ]:
result_rescomm['total rc leak g CH4/Mcf'] = result_rescomm[[
    'prod leak g CH4/Mcf dry gas', 
    'trans leak g CH4/Mcf',
    'dist pipe res-comm leak CH4 g/Mcf 1urb',
    'res-comm meter leak CH4 g/Mcf 1urb',
    'behind-the-meter res-comm CH4 g/Mcf 1urb',
    'add leak res-comm CH4 g/Mcf 1urb rate'
]].sum(axis=1)

## merge in city short names

In [ ]:
city_names = gim_cities.copy()
city_names['metro st'] = city_names['metro area'] + ', ' + city_names['metro area state']
city_names = city_names.rename(columns={'Census urban area name': 'urban name'})
result_rescomm = pd.merge(city_names[['metro st', 'urban name']], result_rescomm, on='urban name', how='left')

In [ ]:
result_rescomm = result_rescomm.sort_values(by='total rc leak g CH4/Mcf', ascending=False)

In [ ]:
# export
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
result_rescomm.to_excel(
    lca_path + f'GIM leakage res-comm by urban area compiled {save_timestamp}.xlsx',
    index=False
)

## end of module